In [1]:
import datetime
from pyspark import SparkConf, SparkContext, HiveContext
sc.stop()
conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "32g")
sc = SparkContext(conf=conf)
#sc.setCheckpointDir('/user/kposminin/checkpointdir/')

In [3]:

def repart(filename):
    starttime = datetime.datetime.now()
    sc.textFile(filename).repartition(32*8).saveAsTextFile('.'.join(filename.split('.')[:-1]))
    print('End. Time of work {0}.'.format(datetime.datetime.now() - starttime))
#repart("/user/kposminin/la_20160824.txt")

End. Time of work 0:01:24.128930.


In [51]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import sklearn
import sklearn.ensemble

# Load and parse the data file.
train = sc.textFile("/user/kposminin/la_20160817") \
    .filter(lambda r: r.count('\t') == 28) \
    .filter(lambda s: (s[0] == '1') or (hash(s) % 10000 == 1)) \
    .map(lambda r:r.split('\t')) \
    .map(lambda r:[int(e) for e in r]) \
    .collect()

test = sc.textFile("/user/kposminin/la_20160824") \
    .filter(lambda r: r.count('\t') == 28) \
    .filter(lambda s: (s[0] == '1') or (int(np.random.rand()*100) == 0)) \
    .map(lambda r:r.split('\t')) \
    .map(lambda r:[int(e) for e in r]) \
    .collect()

test_rdd = sc.textFile("/user/kposminin/la_20160824") \
    .filter(lambda r: r.count('\t') == 28) \
    .map(lambda r:r.split('\t')) \
    .map(lambda r:[int(e) for e in r]) \



### Варьируем размер семплирования  

In [52]:
import numpy as np

modelGBT = {}
AUCROC={}

for f in [1,2,5,10,20]:
    train1 = [r for r in train if (r[0] == 1) or int(np.random.rand()*f) == 0]
    test1 = [r for r in test if (r[0] == 1) or int(np.random.rand()*f) == 0]
    s = len(train1)
    print(s)
    modelGBT[s] = sklearn.ensemble.GradientBoostingClassifier(n_estimators=300, learning_rate=0.1,
       max_depth=6, random_state=0).fit(X = [e[1:] for e in train1], y = [e[0] for e in train1])
    AUCROC[s] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test1], 
        y_score = [r[1] for r in modelGBT[s].predict_proba([e[1:] for e in test1])]
    )
    AUCROC['train '+ str(s)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train1], 
        y_score = [r[1] for r in modelGBT[s].predict_proba([e[1:] for e in train1])]
    )
    AUCROC['smax '+ str(s)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test1], 
        y_score = [e[1] for e in test1]
    )
    AUCROC['train smax '+ str(s)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train1], 
        y_score = [e[1] for e in train1]
    )

17907
9460
4581
3004
2083


In [53]:
len(test)

1745159

In [54]:
for s in modelGBT.keys():
    AUCROC['test_full on {0}-model'.format(s)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test], 
        y_score = [r[1] for r in modelGBT[s].predict_proba([e[1:] for e in test])]
    )

In [55]:
print('AUCROC score from sample number:\n' + '\n'.join(['{0} {1:.5f}'.format(k,v) for (k,v) in sorted(AUCROC.items())]))

AUCROC score from sample number:
2083 0.99163
3004 0.99337
4581 0.99272
9460 0.99299
17907 0.99115
smax 17907 0.85920
smax 2083 0.85927
smax 3004 0.85916
smax 4581 0.85882
smax 9460 0.85926
test_full on 17907-model 0.99115
test_full on 2083-model 0.99104
test_full on 3004-model 0.99309
test_full on 4581-model 0.99193
test_full on 9460-model 0.99240
train 17907 1.00000
train 2083 1.00000
train 3004 1.00000
train 4581 1.00000
train 9460 1.00000
train smax 17907 0.85632
train smax 2083 0.85822
train smax 3004 0.85949
train smax 4581 0.85605
train smax 9460 0.85534


#### Качество классификатора растет при снижении объема выборки вплоть до уровня паритета вероятности классов в выборке.
Выбираем объем обучающей выборки немного с запасом в 9 тыс. примеров. Это соотвествует соотношению классов 5 к 1.


In [69]:
#append avg scores 
def append_param(sample):
    return [r+[sum(r[14:16])/2,sum(r[14:17])/3,sum(r[14:18])/4,sum(r[14:19])/5,sum(r[14:21])/7,(r[14]+r[24])/2] for r in sample]
train = append_param([r for r in train if int(np.random.rand()*2) == 0])
test = append_param(test)

In [79]:
len(train)

8911

In [ ]:
open('test2.txt','w').write(str(AUCROC))

In [4]:
def write_table_to_file(table, filename):
    f = open(filename,'w+')
    #f.write('label,' + ','.join(columns)+'\n')
    f.write('\n'.join([','.join([str(e) for e in r]) for r in table]))
    f.close()

In [30]:
columns = '''smax ,savg ,ssum ,smedian ,sstd ,cntrepeat ,cntuniq 
,duration ,mobile ,emailru ,vkru ,okru ,social_other , s1 ,s2 ,s3 ,s4 ,s5 ,s6 ,s7 ,s8 ,s9 ,s10 , 
sm1 ,sm2 ,sm3 ,sm4 ,sm5'''.replace(' ','').replace('\n','').split(',')

In [ ]:
train = [r for r in train if (r[0] == 1) or int(np.random.rand()*10) == 0]
test = [r for r in test if (r[0] == 1) or int(np.random.rand()*1) == 0]

### Варьируем количество деревьев 

In [71]:
# Train a GradientBoostedTrees model.
import sklearn.ensemble
import sklearn
modelGBT = {}
AUCROC1={}
for n in [20,30,50,80,100,200,300,500,800]:
    modelGBT[n] = sklearn.ensemble.GradientBoostingClassifier(n_estimators=n, learning_rate=0.1,
       max_depth=6, random_state=0).fit(X = [e[1:] for e in train], y = [e[0] for e in train])
    AUCROC1[n] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test], 
        y_score = [r[1] for r in modelGBT[n].predict_proba([e[1:] for e in test])]
    )
    AUCROC1['train '+str(n)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train], 
        y_score = [r[1] for r in modelGBT[n].predict_proba([e[1:] for e in train])]
    )
AUCROC1['smax'] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test], 
        y_score = [e[1] for e in test]
)
AUCROC1['train smax'] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train], 
        y_score = [e[1] for e in train]
)
#print(AUCROC)
# Evaluate model on test instances and compute test error
##predictions = model.predict(test.map(lambda x: x.features))
#predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

In [72]:
print('AUCROC test sample score from GBT trees number:\n' + '\n'.join(['{0} {1:.5f}'.format(k,v) for (k,v) in sorted(AUCROC1.items())]))

AUCROC test sample score from GBT trees number:
20 0.99083
30 0.99142
50 0.99347
80 0.99266
100 0.99240
200 0.98955
300 0.98644
500 0.97686
800 0.97573
smax 0.85920
train 100 1.00000
train 20 0.99841
train 200 1.00000
train 30 0.99890
train 300 1.00000
train 50 0.99972
train 500 1.00000
train 80 0.99998
train 800 1.00000
train smax 0.84172


#### Если брать более 100 деревьев, то происходит переобучение на выборке.
Выбираем 80.

#### Вариация глубины дерева

In [89]:
#sorted(zip(modelGBT[80].feature_importances_,columns))


In [84]:
#modelGBT = {}
#AUCROC1 = {'smax':AUCROC1['smax'], 'train smax':AUCROC1['train smax']}
for m in [1,3,4,5,6,7,8,9,10,11,15]:
    modelGBT[m] = sklearn.ensemble.GradientBoostingClassifier(n_estimators=80, learning_rate=0.1,
       max_depth=m, random_state=0).fit(X = [e[1:] for e in train], y = [e[0] for e in train])
    AUCROC1[m] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test], 
        y_score = [r[1] for r in modelGBT[m].predict_proba([e[1:] for e in test])]
    )
    AUCROC1['train '+str(m)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train], 
        y_score = [r[1] for r in modelGBT[m].predict_proba([e[1:] for e in train])]
    )


In [85]:
print('AUCROC test sample score from GBT tree depth:\n' + '\n'.join(['{0} {1:.5f}'.format(k,v) for (k,v) in sorted(AUCROC1.items())]))

AUCROC test sample score from GBT tree depth:
1 0.99055
3 0.99292
4 0.99108
5 0.99282
6 0.99240
7 0.99263
8 0.99152
9 0.99073
10 0.99075
11 0.98913
15 0.98571
smax 0.85920
train 1 0.98673
train 10 1.00000
train 11 1.00000
train 15 1.00000
train 3 0.99752
train 4 0.99943
train 5 0.99992
train 6 1.00000
train 7 1.00000
train 8 1.00000
train 9 1.00000
train smax 0.84172


#### До глубины 7 разница несущественна, далее начинается переобучение.
Выбираем глубину в 6 нодов.

In [88]:
model = modelGBT[6]
import pickle
pickle.dump(model,open('modelGBT.pck','w'))

In [93]:
sorted(zip(model.feature_importances_,columns))
sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test], 
        y_score = [r[1] for r in model.predict_proba([e[1:] for e in test])]
)

0.99239522347437292

In [94]:
res = sorted([(r[0],int(10**4 * model.predict_proba(r[1:])[0][1])) for r in test],key = lambda r: -r[1])

In [101]:
postot = sum([r[0] for r in res])
len(res),postot


(1745159, 1135)

In [107]:
print('Share of positives in sample top quantile\nquantile\tpositive rate\tclass share')
for q in [0.001,0.005,0.01,0.02,0.03,0.05,0.1]:
    t = int(len(res) * q)
    p = sum([r[0] for r in res[:t]])
    print('{0}\t{1:.2%}\t{2:.2%}'.format(q,1.*p/postot,p*1./(t-p)))

Share of positives in sample top quantile
quantile	positive rate	class share
0.001	60.53%	64.93%
0.005	82.56%	12.03%
0.01	86.96%	5.99%
0.02	91.19%	3.06%
0.03	93.57%	2.07%
0.05	96.30%	1.27%
0.1	98.41%	0.64%


In [ ]:
res1 = test_rdd.map(lambda r:(float(r[0]),model.predict_proba(r[1:])[0][1])).collect()
ar = sklearn.metrics.roc_auc_score(y_true = [e[0] for e in res1], y_score = [e[1] for e in res1])

Py4JJavaError: An error occurred while calling o28.defaultParallelism.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:234)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:381)
py4j.Gateway.invoke(Gateway.java:214)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:79)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:68)
py4j.GatewayConnection.run(GatewayConnection.java:209)
java.lang.Thread.run(Thread.ja

In [108]:
print('Model feature importances\n'+'\n'.join(['{0} {1:.5f}'.format(k,v) for (k,v) in sorted(zip(columns,model.feature_importances_), key = lambda r:-r[1])]))

Model feature importances
smax 0.11443
s1 0.11212
ssum 0.07378
duration 0.06721
sm1 0.05852
sstd 0.05731
cntrepeat 0.05340
savg 0.05099
sm5 0.02901
smedian 0.02864
s3 0.02492
s4 0.02281
s2 0.02069
sm2 0.01928
s10 0.01853
s5 0.01323
cntuniq 0.01192
s6 0.01077
s8 0.01075
sm3 0.01050
vkru 0.00992
sm4 0.00879
s7 0.00876
s9 0.00752
emailru 0.00590
okru 0.00163
mobile 0.00148
social_other 0.00004


In [3]:
r = sc.parallelize(range(1000)).map(lambda r: (1,r))

In [4]:
prob = [0.5,0.6,0.7,0.8,0.9,0.95,0.97,0.98,0.99,1]
res = r \
    .map(lambda r:[r[0] * (r[1]>p*1000) for p in prob]) \
    .reduce(lambda x,y:[x[i]+y[i] for i in range(len(x))])
print(res)

[499, 399, 299, 199, 99, 49, 29, 19, 9, 0]
